# Dask Array

Материалы:
* Макрушин С.В. Лекция 11: Dask
* https://docs.dask.org/en/latest/array.html
* JESSE C. DANIEL. Data Science with Python and Dask. 

## Задачи для совместного разбора

1. Создайте массив размерностью 1000 на 300000, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики полученного массива.

In [4]:
import dask.array as da
import numpy as np

In [5]:
import dask
dask.__version__

'2.20.0'

In [4]:
%%time
arr_np = np.random.normal(0,1,size=(1_000,300_000))

CPU times: user 8.23 s, sys: 666 ms, total: 8.9 s
Wall time: 9.01 s


In [6]:
%%time
arr_da = da.random.normal(0,1,size=(1_000,300_000), chunks=(1_000, 30_000))

CPU times: user 3.52 ms, sys: 3.17 ms, total: 6.69 ms
Wall time: 10.9 ms


In [7]:
arr_da

dask.array<normal, shape=(1000, 300000), dtype=float64, chunksize=(1000, 30000), chunktype=numpy.ndarray>

2. Посчитайте сумму квадратов элементов массива, созданного в задаче 1. Создайте массив `np.array` такого же размера и сравните скорость решения задачи с использование `da.array` и `np.array`

In [9]:
%%time
np.power(arr_np, 2).sum()

CPU times: user 8.4 s, sys: 882 ms, total: 9.28 s
Wall time: 9.3 s


300004889.5001061

In [12]:
%%time
da.power(arr_da, 2).sum().compute()

CPU times: user 26.4 s, sys: 2.17 s, total: 28.6 s
Wall time: 8.86 s


300032249.3385624

3. Визуализируйте граф вычислений для задачи 2.

## Лабораторная работа 11

In [3]:
import dask.array as da
import h5py
import numpy as np

1. Считайте датасет `recipe` из файла `minutes_n_ingredients_full.hdf5` в виде `dask.array`. Укажите аргумент `chunks=(100_000, 3)` при создании массива. Выведите на экран основную информацию о массиве.

In [4]:
import h5py
f = h5py.File('minutes_n_ingredients_full.hdf5', 'r') # HDF5 file
d = f['recipe']          # Pointer on on-disk array

In [5]:
d

<HDF5 dataset "recipe": shape (2231637, 3), type "<i8">

In [5]:
import dask.array as da
x = da.from_array(d, chunks=(100_000, 3))
x

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(100000, 3), chunktype=numpy.ndarray>

2. Вычислите среднее значение по каждому столбцу, кроме первого. 

In [54]:
r = x[:,1:].mean(axis=0).compute()
r

array([1004.20805176,    5.4198008 ])

3. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

In [15]:
x1 = da.from_array(d)
x2 = da.from_array(d, chunks=(100_000, 3))
x3 = da.from_array(d, chunks=(100_000, 4))
x4 = da.from_array(d, chunks=(200_000, 4))

In [16]:
%%time
r1 = x1[:,1:].mean(axis=0).compute()
r1

CPU times: user 116 ms, sys: 27.6 ms, total: 144 ms
Wall time: 150 ms


array([1004.20805176,    5.4198008 ])

In [17]:
%%time
r2 = x2[:,1:].mean(axis=0).compute()
r2

CPU times: user 173 ms, sys: 21.2 ms, total: 194 ms
Wall time: 121 ms


array([1004.20805176,    5.4198008 ])

In [18]:
%%time
r3 = x3[:,1:].mean(axis=0).compute()
r3

CPU times: user 170 ms, sys: 18.4 ms, total: 188 ms
Wall time: 114 ms


array([1004.20805176,    5.4198008 ])

In [19]:
%%time
r4 = x4[:,1:].mean(axis=0).compute()
r4

CPU times: user 165 ms, sys: 23.8 ms, total: 189 ms
Wall time: 135 ms


array([1004.20805176,    5.4198008 ])

С chunks время выполнения занимает больше. Чем больше chunks, тем больше время выполнения

4. Выберите рецепты, время выполнения которых меньше медианного значения

In [7]:
# Посмотрим имена столбцов
with h5py.File('minutes_n_ingredients_full.hdf5', 'r') as f:
    for key in f.keys():
        vals = [val for val in f[key].attrs.values()]
        print(vals)

['Содержит столбцы id, minutes и n_ingredients из recipes_full.csv']


In [19]:
med = da.median(x, axis=0).compute()
med = med[1]
print(med)

32.0


In [20]:
x4 = x[x[:,1] < med].compute()

In [21]:
x4

array([[1089012,      23,       5],
       [1428572,       0,       5],
       [1400250,      24,       1],
       ...,
       [1029131,      19,       4],
       [1700703,       1,       1],
       [ 713836,       0,       9]])

In [22]:
print(len(x), len(x4))


2231637 1084304


5. Посчитайте количество каждого из возможных значений кол-ва ингредиентов

In [34]:
sp = x[:,2].compute()

In [35]:
x5 = da.unique(x[:,2], return_counts=True)

In [36]:
# каике количества встретились
x5[0].compute()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 43])

In [37]:
# сколько раз встретились
x5[1].compute()

array([222071, 224158, 229388, 234948, 240720, 244360, 247181, 246747,
       246816,  22430,  19094,  15165,  11640,   8284,   6014,   4145,
         2793,   1913,   1279,    852,    529,    346,    244,    178,
          107,     68,     55,     33,     22,     20,     13,      5,
            4,      3,      4,      1,      2,      1,      1,      2,
            1])

In [41]:
dt = dict(zip(x5[0].compute(), x5[1].compute()))
for k,v in dt.items():
    print(k, 'встретилось', v, 'раз')

1 встретилось 222071 раз
2 встретилось 224158 раз
3 встретилось 229388 раз
4 встретилось 234948 раз
5 встретилось 240720 раз
6 встретилось 244360 раз
7 встретилось 247181 раз
8 встретилось 246747 раз
9 встретилось 246816 раз
10 встретилось 22430 раз
11 встретилось 19094 раз
12 встретилось 15165 раз
13 встретилось 11640 раз
14 встретилось 8284 раз
15 встретилось 6014 раз
16 встретилось 4145 раз
17 встретилось 2793 раз
18 встретилось 1913 раз
19 встретилось 1279 раз
20 встретилось 852 раз
21 встретилось 529 раз
22 встретилось 346 раз
23 встретилось 244 раз
24 встретилось 178 раз
25 встретилось 107 раз
26 встретилось 68 раз
27 встретилось 55 раз
28 встретилось 33 раз
29 встретилось 22 раз
30 встретилось 20 раз
31 встретилось 13 раз
32 встретилось 5 раз
33 встретилось 4 раз
34 встретилось 3 раз
35 встретилось 4 раз
36 встретилось 1 раз
37 встретилось 2 раз
38 встретилось 1 раз
39 встретилось 1 раз
40 встретилось 2 раз
43 встретилось 1 раз


In [122]:
# решение с d
dict_n_ing = {}
for i in d:
    if i[2] not in dict_n_ing:
        count = da.where(d[:,2] != i[2], 0, 1).sum().compute()
        print(i[2], 'встретилось', count, 'раз')
        dict_n_ing[i[2]] = count

9 встретилось 246816 раз
5 встретилось 240720 раз
1 встретилось 222071 раз
10 встретилось 22430 раз
6 встретилось 244360 раз
4 встретилось 234948 раз
8 встретилось 246747 раз
3 встретилось 229388 раз
2 встретилось 224158 раз
7 встретилось 247181 раз
13 встретилось 11640 раз
15 встретилось 6014 раз
11 встретилось 19094 раз
16 встретилось 4145 раз
19 встретилось 1279 раз
14 встретилось 8284 раз
12 встретилось 15165 раз
18 встретилось 1913 раз
29 встретилось 22 раз
17 встретилось 2793 раз
20 встретилось 852 раз
22 встретилось 346 раз
21 встретилось 529 раз
26 встретилось 68 раз
25 встретилось 107 раз
23 встретилось 244 раз
28 встретилось 33 раз
32 встретилось 5 раз
24 встретилось 178 раз
27 встретилось 55 раз
35 встретилось 4 раз
31 встретилось 13 раз
33 встретилось 4 раз
43 встретилось 1 раз
39 встретилось 1 раз
37 встретилось 2 раз
30 встретилось 20 раз
40 встретилось 2 раз
34 встретилось 3 раз
38 встретилось 1 раз
36 встретилось 1 раз


6. Найдите максимальную продолжительность рецепта. Ограничьте максимальную продолжительность рецептов сверху значением, равному 75% квантилю.

In [44]:
import pandas as pd

In [52]:
xq = pd.Series(x[:,1].compute())
q075 = xq.quantile(q=0.75)
q075

49.0

In [57]:
b = x[x[:,1] < q075].compute()

In [56]:
b[:10]

array([[ 683970,      33,       9],
       [1089012,      23,       5],
       [1428572,       0,       5],
       [1400250,      24,       1],
       [ 387709,      47,      10],
       [1798295,      29,       5],
       [ 814242,      37,       5],
       [ 818815,      21,       5],
       [ 357565,      32,       6],
       [1121713,      11,       4]])

In [54]:
minutes = x[:,1].compute()
minutes[:10]

array([33, 23,  0, 24, 47, 29, 60, 37, 21, 57])

Как видно из проверки 60 и 57 нет в ответе

7. Создайте массив `dask.array` из 2 чисел, содержащих ваши предпочтения относительно времени выполнения рецепта и кол-ва ингредиентов. Найдите наиболее похожий (в смысле $L_1$) рецепт из имеющихся в датасете.

In [10]:
from nltk.metrics.distance import edit_distance
import pandas as pd

In [11]:
def dist_x(x7):
    x_min = pd.Series(x[:,1].compute())
    x_n_ing = pd.Series(x[:,2].compute())
    sp = []
    tf = True
    for i in range(len(x_min)):
        s = [x_min[i],x_n_ing[i]]
        dist = edit_distance(s, x7)
        if dist == 0:
            return 'Найден наиболее похожий рецепт:', x[i].compute()
        if dist == 1:
            sp.append(x[i].compute())
    if len(sp) == 0:
        return('Не найдено ни одного похожего рецепта')
    else:
        return('Найдены приблизительно похожие рецепты:', sp)


In [12]:
x7 = da.from_array([7,4])
x7

dask.array<array, shape=(2,), dtype=int64, chunksize=(2,), chunktype=numpy.ndarray>

In [15]:
dist_x(x7)

('Найден наиболее похожий рецепт:', array([838857,      7,      4]))

8. Работая с исходным файлом в формате `hdf5`, реализуйте алгоритм подсчета среднего значения в блочной форме и вычислите с его помощью среднее значение второго столбца в массиве.

Блочный алгоритм вычислений состоит из двух частей:
1. загрузка фрагмента за фрагментом данных по `blocksize` элементов и проведение вычислений на этим фрагментом;
2. агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных.

Важно: при работе с `h5py` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент

In [7]:
x.numblocks[0]

23

In [8]:
x.shape[0]

2231637

In [10]:
sums = 0
for block_num in range(x.numblocks[0]):
    sums += da.sum(x.blocks[block_num][:,1])
(sums / x.shape[0]).compute()

1004.2080517575215

In [13]:
# проверяем, что всё посчитано верно
da.mean(x[:,1]).compute()

1004.2080517575215